In [1]:
from training.model import SIREN
import torch
import numpy as np
from render.meshing import create_mesh
import open3d as o3d
import json
import meshplot as mp
import matplotlib.cm as cm

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
MESH = 'faust_54098'
EXPERIMENT = 'faust_54098'
N = 32
MODEL = 'best'


meshes_pred = []
meshes_gt = []
with open(f'../results/{MESH}/{MESH}.json') as jsonFile:
    skel = json.load(jsonFile)
    model = SIREN(
            n_in_features= len(skel['joints'][0]['mean']) + 3,
            n_out_features=1,
            hidden_layer_config=[256, 256, 256, 256],
            w0=30,
            ww=None
    )
    model.load_state_dict( torch.load(f'../results/{MESH}/test_{EXPERIMENT}/models/model_{MODEL}.pth', map_location=torch.device(0)))
    for joint in skel['joints']:
        vertices, faces, normals, values = create_mesh(
            model,
            distance = torch.from_numpy(np.array(joint['mean']).astype(np.float32)),
            N=N
        )
        mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(vertices), o3d.utility.Vector3iVector(faces) )
        mesh.transform( np.array(joint['transformation']).astype(np.float32) )
        meshes_pred.append(mesh)

        mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(np.array(joint['vertices']).astype(np.float32)), o3d.utility.Vector3iVector(np.array(joint['triangles']).astype(np.float32)) )
        mesh.transform( np.array(joint['transformation']).astype(np.float32) )
        meshes_gt.append(mesh)


RuntimeError: don't know how to restore data location of torch.storage._UntypedStorage (tagged with CUDA:0)

In [ ]:

p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_pred):
    if len(np.asarray(mesh.vertices)) > 0:
        if p is None:
            p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_pred)))[0:3])
        else:
            p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_pred)))[0:3] )

In [ ]:

p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_gt):
    if p is None:
        p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_gt)))[0:3])
    else:
        p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_gt)))[0:3] )

In [ ]:
MESH = 'brazo'
EXPERIMENT = 'brazo'
BRANCH = 0
N = 64
MODEL = 'best'

model = SIREN(
        n_in_features=4,
        n_out_features=1,
        hidden_layer_config=[256, 256, 256, 256],
        w0=30,
        ww=None
)
model.load_state_dict( torch.load(f'../results/{MESH}/test_{EXPERIMENT}/models/model_{MODEL}.pth'))

meshes_pred = []
with open(f'../results/{MESH}/{MESH}.json') as jsonFile:
    skel = json.load(jsonFile)
    branch = skel['branches'][BRANCH]

    for d in np.linspace(0,1,6):
        vertices, faces, normals, values = create_mesh(
            model,
            distance = torch.from_numpy(np.array(d).astype(np.float32)),
            N=N
        )
        mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(vertices), o3d.utility.Vector3iVector(faces) )
        mesh.transform( .astype(np.float32) )
        meshes_pred.append(mesh)

In [ ]:

p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_pred):
    if p is None:
        p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_pred)))[0:3])
    else:
        p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_pred)))[0:3] )